In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import ProjectedGradientDescent

# Suppress warnings
warnings.filterwarnings('ignore')
print("All libraries imported successfully.")

2025-11-01 16:54:39.276269: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/reshma/Documents/Coding/capstone_project/attack/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/reshma/Documents/Coding/capstone_project/attack/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/reshma/Documents/Coding/capstone_project/attack/lib/python3.9/site-packages/art/estimators/cert

All libraries imported successfully.


In [ ]:
DATA_PATH = 'CIC-Darknet2020.csv'
TARGET_LABELS = ['Tor', 'Non-Tor', 'VPN', 'NonVPN']

MODEL_PATH = 'model-defended-v2.h5'
SCALER_PATH = 'scaler-multi.pkl'

PGD_EPS = 0.1
PGD_EPS_STEP = 0.01 
PGD_MAX_ITER = 40
PGD_BATCH_SIZE = 64

In [ ]:
def load_multi_class_test_data():
    try:
        df = pd.read_csv(DATA_PATH)
    except FileNotFoundError:
        print(f"Error: '{DATA_PATH}' not found.")
        return None

    df.columns = [*df.columns[:-2], 'Label', 'Label_Type']
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    df_multi = df[df['Label'].isin(TARGET_LABELS)].copy()

    non_feature_cols = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Label', 'Label_Type']
    X = df_multi.drop(columns=non_feature_cols).apply(pd.to_numeric)
    y = df_multi['Label']

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    n_classes = len(le.classes_)
    target_names = le.classes_
    y_ohe = to_categorical(y_encoded, num_classes=n_classes)
    
    print("--- Class Encoding Mapping ---")
    for index, label in enumerate(le.classes_):
        print(f"Class Index {index} -> {label}")

    X_train, X_test, y_train_ohe, y_test_ohe, y_train_encoded, y_test_encoded = train_test_split(
        X, y_ohe, y_encoded,
        test_size=0.2,
        random_state=42,
        stratify=y_encoded
    )

    try:
        scaler = joblib.load(SCALER_PATH)
        print(f"\nScaler '{SCALER_PATH}' loaded successfully.")
    except FileNotFoundError:
        print(f"Error: '{SCALER_PATH}' not found. Did you run the baseline notebook?")
        return None
        
    X_test_scaled = scaler.transform(X_test)

    n_features = X_test_scaled.shape[1]
    X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], n_features, 1))
    
    print(f"Data preparation complete. Found {n_features} features and {n_classes} classes.")
    
    return X_test_cnn, y_test_ohe, y_test_encoded, target_names, n_features, n_classes

In [ ]:
def get_art_classifier_multi(model_path, n_features, n_classes):
    try:
        model = load_model(model_path)
        print(f"Model '{model_path}' loaded successfully.")
    except Exception as e:
        print(f"Error: Model file '{model_path}' not found or failed to load.")
        print(f"Details: {e}")
        return None, None
        
    loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

    classifier = TensorFlowV2Classifier(
        model=model,
        loss_object=loss_object,
        input_shape=(n_features, 1),
        nb_classes=n_classes, 
        channels_first=False
    )
    
    return model, classifier

In [ ]:
def evaluate_attack_multi(y_true_encoded, y_pred_probs, attack_name, target_names):
    y_pred_encoded = np.argmax(y_pred_probs, axis=1)
    
    acc = accuracy_score(y_true_encoded, y_pred_encoded)
    
    print(f"\n--- {attack_name} Metrics ---")
    print(f"Accuracy: {acc * 100:.4f}%")
    
    print("\nClassification Report:")
    print(classification_report(y_true_encoded, y_pred_encoded, target_names=target_names))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true_encoded, y_pred_encoded))
    
    return acc

In [ ]:
data = load_multi_class_test_data()

if data:
    X_test_cnn, y_test_ohe, y_test_encoded, target_names, n_features, n_classes = data
    
    X_test_art = X_test_cnn.astype(np.float32)

    model, classifier = get_art_classifier_multi(MODEL_PATH, n_features, n_classes)
else:
    print("Data loading failed. Cannot proceed.")

--- Class Encoding Mapping ---
Class Index 0 -> Non-Tor
Class Index 1 -> NonVPN
Class Index 2 -> Tor
Class Index 3 -> VPN

Scaler 'scaler-multi.pkl' loaded successfully.
Data preparation complete. Found 76 features and 4 classes.


Model 'model-defended-v2.h5' loaded successfully.


In [ ]:
if 'model' in locals():
    print("\nEvaluating baseline (clean) model performance...")
    y_pred_clean_probs = model.predict(X_test_art)
    
    clean_acc = evaluate_attack_multi(y_test_encoded, y_pred_clean_probs, 
                                      "Baseline (Clean)", target_names)
else:
    print("Model not loaded. Skipping baseline evaluation.")


Evaluating baseline (clean) model performance...
992/992 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step

--- Baseline (Clean) Metrics ---
Accuracy: 95.9135%

Classification Report:
              precision    recall  f1-score   support

     Non-Tor       1.00      0.99      1.00     22079
      NonVPN       0.88      0.86      0.87      4772
         Tor       0.90      0.87      0.89       279
         VPN       0.86      0.90      0.88      4584

    accuracy                           0.96     31714
   macro avg       0.91      0.91      0.91     31714
weighted avg       0.96      0.96      0.96     31714


Confusion Matrix:
[[21911   120     1    47]
 [   29  4120    22   601]
 [    0    23   244    12]
 [   12   426     3  4143]]


In [ ]:
if 'classifier' in locals() and 'clean_acc' in locals():
    print("\n" + "="*50)
    print(f"Starting PGD Attack (Epsilon = {PGD_EPS}, Iterations = {PGD_MAX_ITER})")
    print("="*50 + "\n")

    attack = ProjectedGradientDescent(
        classifier,
        eps=PGD_EPS,
        eps_step=PGD_EPS_STEP,
        max_iter=PGD_MAX_ITER,
        batch_size=PGD_BATCH_SIZE,
        verbose=True
    )

    print(f"Generating adversarial examples...")
    X_test_adv = attack.generate(x=X_test_art, y=y_test_ohe)
    
    print("\nEvaluating model on Adversarial Examples...")
    y_pred_adv_probs = model.predict(X_test_adv)
    
    adv_acc = evaluate_attack_multi(y_test_encoded, y_pred_adv_probs, 
                                    f"PGD Attack (eps={PGD_EPS})", target_names)
    
    print("\n" + "="*50)
    print(f"Baseline (Clean) Accuracy: {clean_acc * 100:.4f}%")
    print(f"Adversarial (PGD) Accuracy: {adv_acc * 100:.4f}%")
    print(f"Accuracy Drop: {(clean_acc - adv_acc) * 100:.4f}%")
    print("="*50)

else:
    print("Classifier or clean accuracy not found. Skipping attack.")


Starting PGD Attack (Epsilon = 0.1, Iterations = 40)

Generating adversarial examples...


PGD - Batches: 496it [19:34,  2.13s/it]2025-11-01 17:15:30.879115: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence



Evaluating model on Adversarial Examples...
992/992 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step

--- PGD Attack (eps=0.1) Metrics ---
Accuracy: 1.0185%

Classification Report:
              precision    recall  f1-score   support

     Non-Tor       0.25      0.01      0.01     22079
      NonVPN       0.00      0.01      0.00      4772
         Tor       0.25      0.52      0.33       279
         VPN       0.00      0.00      0.00      4584

    accuracy                           0.01     31714
   macro avg       0.12      0.14      0.09     31714
weighted avg       0.17      0.01      0.01     31714


Confusion Matrix:
[[  124 11607    14 10334]
 [  252    48   431  4041]
 [   18    72   146    43]
 [  107  4468     4     5]]

Baseline (Clean) Accuracy: 95.9135%
Adversarial (PGD) Accuracy: 1.0185%
Accuracy Drop: 94.8950%
